In [3]:
from scipy import stats
from scipy.stats import skew,norm  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.stats as stats


#Visualizing tools
import seaborn as sns
import matplotlib.pyplot as plt

#preprocessing tools
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

#ML Algoirthm
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
import sklearn.linear_model as linear_model
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
ttrain = pd.read_csv('Train (1).csv')
ttest = pd.read_csv('Test (1).csv')

In [5]:
fin_data = pd.concat([ttrain,ttest]).reset_index(drop=True)

In [6]:
fin_data.head()

,ID,country,year,urban_or_rural,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline,Target
0,ID_AAIethGy,Ethiopia,2016,R,0.0,0.000000,0.000000,0.000055,0.000536,0.999408,12.146134,25.489659,0.879484,0.000000,0.000000,0.000000,278.788451,769.338378,0.132783
1,ID_AAYiaCeL,Ethiopia,2005,R,0.0,0.000000,0.000110,0.000000,0.000018,0.999872,113.806716,64.136053,0.601427,0.000000,0.005427,0.000000,200.986978,337.135243,0.004898
2,ID_AAdurmKj,Mozambique,2009,R,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4.400096,0.131900,0.000000,0.003078,0.000000,642.594208,169.913773,0.097320
3,ID_AAgNHles,Malawi,2015,R,0.0,0.000141,0.000181,0.000254,0.000228,0.999195,5.213320,25.379371,2.017136,11.293841,0.131035,0.000000,365.349451,613.591610,0.304107
4,ID_AAishfND,Guinea,2012,U,0.0,0.011649,0.017560,0.017383,0.099875,0.853533,31.734661,5.081620,22.815984,0.005047,0.130475,1.461894,222.867189,192.926363,0.605328


In [7]:
fin_data.isnull().sum()

ID                                            0
country                                       0
year                                          0
urban_or_rural                                0
ghsl_water_surface                            0
ghsl_built_pre_1975                           0
ghsl_built_1975_to_1990                       0
ghsl_built_1990_to_2000                       0
ghsl_built_2000_to_2014                       0
ghsl_not_built_up                             0
ghsl_pop_density                              0
landcover_crops_fraction                      0
landcover_urban_fraction                      0
landcover_water_permanent_10km_fraction       0
landcover_water_seasonal_10km_fraction        0
nighttime_lights                              0
dist_to_capital                               0
dist_to_shoreline                             0
Target                                     7194
dtype: int64

In [8]:
fin_data.drop(['ID'], axis=1, inplace = True)

In [9]:
fin_data.describe()

,year,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline,Target
count,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,28648.000000,21454.000000
mean,2010.271537,0.027018,0.041010,0.027820,0.013713,0.018369,0.872069,95.101508,21.097237,14.079153,1.668968,0.694652,9.260498,300.584775,473.801360,0.350736
std,4.553145,0.095199,0.118721,0.072850,0.034208,0.040241,0.242707,210.079701,16.309999,23.917510,7.152657,2.383790,26.160952,269.097712,355.856845,0.194376
min,1994.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000859,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105306,0.112080,0.000000
25%,2008.000000,0.000000,0.000000,0.000000,0.000037,0.000109,0.895615,2.943082,6.105505,0.705307,0.000000,0.001394,0.000000,110.789483,158.829182,0.195772
50%,2011.000000,0.000000,0.000160,0.000602,0.000936,0.001727,0.992725,16.424158,19.297194,2.599028,0.000579,0.033941,0.024074,253.537926,420.442368,0.293574
75%,2014.000000,0.000000,0.008011,0.010037,0.008823,0.015209,0.999592,64.364772,33.199903,13.063126,0.172243,0.375420,4.963472,403.780077,732.552186,0.499003
max,2016.000000,0.991100,0.894608,0.685010,0.515534,0.649159,1.000000,1902.876955,80.064918,98.784092,99.164018,56.201637,403.455783,1908.105037,1769.523906,1.000000


In [10]:
fin_data.corr()

,year,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline,Target
year,1.000000,-0.012947,-0.083721,-0.038382,-0.018690,0.017530,0.057279,-0.048618,0.082205,-0.055750,0.034326,-0.054697,-0.028213,0.019710,-0.015886,0.115014
ghsl_water_surface,-0.012947,1.000000,0.197961,0.110012,0.040389,0.034900,-0.533573,0.137382,-0.221121,0.246438,0.639638,0.185985,0.171599,-0.030484,-0.152016,0.182816
ghsl_built_pre_1975,-0.083721,0.197961,1.000000,0.474982,0.464407,0.281641,-0.821521,0.816594,-0.337366,0.804210,0.079085,0.053655,0.746656,-0.147007,-0.186005,0.450119
ghsl_built_1975_to_1990,-0.038382,0.110012,0.474982,1.000000,0.451552,0.445210,-0.713104,0.658810,-0.298848,0.750170,0.042565,0.070884,0.567003,-0.186747,-0.151022,0.500753
ghsl_built_1990_to_2000,-0.018690,0.040389,0.464407,0.451552,1.000000,0.574375,-0.614719,0.507582,-0.257735,0.638640,0.017545,0.017371,0.414382,-0.166710,-0.142295,0.425059
ghsl_built_2000_to_2014,0.017530,0.034900,0.281641,0.445210,0.574375,1.000000,-0.531842,0.416470,-0.226773,0.587058,0.003370,0.023246,0.355522,-0.167433,-0.088179,0.429262
ghsl_not_built_up,0.057279,-0.533573,-0.821521,-0.713104,-0.614719,-0.531842,1.000000,-0.791663,0.415382,-0.902559,-0.305384,-0.126775,-0.720076,0.191176,0.230617,-0.580670
ghsl_pop_density,-0.048618,0.137382,0.816594,0.658810,0.507582,0.416470,-0.791663,1.000000,-0.323615,0.860147,0.044945,0.038811,0.761638,-0.162155,-0.164513,0.525206
landcover_crops_fraction,0.082205,-0.221121,-0.337366,-0.298848,-0.257735,-0.226773,0.415382,-0.323615,1.000000,-0.377178,-0.112033,-0.096018,-0.304454,-0.055470,0.281731,-0.253550
landcover_urban_fraction,-0.055750,0.246438,0.804210,0.750170,0.638640,0.587058,-0.902559,0.860147,-0.377178,1.000000,0.059166,0.066508,0.761303,-0.214692,-0.214850,0.666768


In [11]:
fin_data['ghsl_built_pre_1975'].value_counts()

0.000000    11051
0.301812       45
0.020879       10
0.023733        9
0.040677        7
            ...  
0.035938        1
0.011718        1
0.000409        1
0.000222        1
0.000017        1
Name: ghsl_built_pre_1975, Length: 17431, dtype: int64

In [12]:
## Too many zeros, drop 

In [13]:
fin_data['ghsl_water_surface'].value_counts()

0.000000    22466
0.090618       45
0.098464        6
0.010934        5
0.019959        5
            ...  
0.000019        1
0.000036        1
0.159987        1
0.016689        1
0.611528        1
Name: ghsl_water_surface, Length: 6114, dtype: int64

In [14]:
fin_data['nighttime_lights'].value_counts()

0.000000     14027
12.697095       45
0.686495        10
1.581286         9
2.148200         7
             ...  
0.275404         1
1.729917         1
0.229549         1
53.345961        1
7.173305         1
Name: nighttime_lights, Length: 14491, dtype: int64

In [15]:
fin_data['landcover_water_permanent_10km_fraction'].value_counts()

0.000000    13318
4.869094       45
0.081717       10
0.376976        9
0.242532        7
            ...  
0.000579        1
0.304364        1
0.002604        1
5.128968        1
0.006292        1
Name: landcover_water_permanent_10km_fraction, Length: 15203, dtype: int64

In [16]:
#fin_data.drop(['ghsl_built_pre_1975'], axis=1, inplace = True)

In [17]:
fin_data.drop(['ghsl_water_surface'], axis=1, inplace = True)

In [18]:
import sklearn
numeric = fin_data.select_dtypes(include = 'number').columns
skew_features = fin_data[numeric].apply(lambda x: skew(x)).sort_values(ascending = False)

from pandas import DataFrame
high_skew = skew_features[skew_features > 6]
skew_index = high_skew.index
print("There are {} numerical features with Skew > 0.5:".format(high_skew.shape[0]))
Skewness = pd.DataFrame({'Skew':high_skew})
Skewness.head(12)

There are 3 numerical features with Skew > 0.5:


,Skew
landcover_water_seasonal_10km_fraction,8.674108
landcover_water_permanent_10km_fraction,6.798676
nighttime_lights,6.292446


In [19]:
from sklearn.preprocessing import PowerTransformer
log = PowerTransformer()
for i in Skewness.index:
    fin_data[i] = log.fit_transform(fin_data[[i]])
    print(i)

landcover_water_seasonal_10km_fraction
landcover_water_permanent_10km_fraction
nighttime_lights


In [20]:
fin_data = pd.get_dummies(fin_data, columns=['urban_or_rural'], drop_first = True)
fin_data['year'] = fin_data['year'].astype('string')

In [21]:
fin_data

,country,year,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline,Target,urban_or_rural_U
0,Ethiopia,2016,0.000000,0.000000,0.000055,0.000536,0.999408,12.146134,25.489659,0.879484,-0.620288,-0.822731,-0.855979,278.788451,769.338378,0.132783,0
1,Ethiopia,2005,0.000000,0.000110,0.000000,0.000018,0.999872,113.806716,64.136053,0.601427,-0.620288,-0.782017,-0.855979,200.986978,337.135243,0.004898,0
2,Mozambique,2009,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4.400096,0.131900,-0.620288,-0.799539,-0.855979,642.594208,169.913773,0.097320,0
3,Malawi,2015,0.000141,0.000181,0.000254,0.000228,0.999195,5.213320,25.379371,2.017136,2.138132,-0.028879,-0.855979,365.349451,613.591610,0.304107,0
4,Guinea,2012,0.011649,0.017560,0.017383,0.099875,0.853533,31.734661,5.081620,22.815984,-0.585709,-0.031571,0.372093,222.867189,192.926363,0.605328,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28643,Zimbabwe,2010,0.000000,0.000000,0.000016,0.000131,0.999853,0.304446,32.263180,0.658724,-0.620288,-0.309046,-0.682316,500.988790,703.139988,NaN,0
28644,Uganda,2011,0.000056,0.000883,0.003861,0.002483,0.992717,36.160182,27.520555,3.615442,-0.620288,-0.813981,-0.855979,159.587852,1000.194893,NaN,1
28645,Burkina Faso,2010,0.000268,0.002800,0.009322,0.042527,0.942399,46.127465,35.674511,12.705561,-0.612497,1.338286,1.012941,33.775441,781.833777,NaN,1
28646,Zimbabwe,2011,0.000000,0.000017,0.000000,0.000100,0.999551,0.309696,18.050789,0.030514,-0.403674,0.352797,-0.578246,320.974153,605.920204,NaN,0


In [22]:
fin_data['dist_to_capital'] = fin_data['dist_to_capital'].astype('float')

In [23]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler,StandardScaler
scaler = MinMaxScaler()
#scaler = StandardScaler()
#fin_data['dist_to_shoreline'] = pd.DataFrame(scaler.fit_transform(fin_data['dist_to_shoreline'].values.reshape(-1,1)))
#fin_data['dist_to_capital'] = pd.DataFrame(scaler.fit_transform(fin_data['dist_to_capital'].values.reshape(-1,1)))

for i in fin_data.drop(['country','year'], axis=1).columns:
    fin_data[i] = pd.DataFrame(scaler.fit_transform(fin_data[i].values.reshape(-1,1)))

In [24]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
    
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [25]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits = 5, shuffle = True, random_state=99)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [26]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [27]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state=99)
split=ttrain.shape[0]
train_df=fin_data[:split]
test_df=fin_data[split:]

In [28]:
train_feat, test_feat = mean_target_encoding(train_df, test_df, 'Target', 'country', alpha=5)

C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [29]:
train_df['country'] = train_feat
test_df['country'] = test_feat

C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
train_feat, test_feat = mean_target_encoding(train_df, test_df, 'Target', 'year', alpha=5)

C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [31]:
train_df['year'] = train_feat
test_df['year'] = test_feat

C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
import lightgbm as lgb
import warnings

from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot

from catboost import CatBoostRegressor

In [33]:
X = train_df.drop(['Target'],axis =1)
y = train_df['Target']
train_labels = y

test_df = test_df.drop(['Target'],axis =1)

In [49]:
fold_score = 0
test_pred = []

n = 8
#kf = StratifiedKFold(n)
kf = KFold(n_splits=25)
#kf = StratifiedShuffleSplit(n_splits=20,test_size=0.2)
kf.get_n_splits(X)
  

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    Xtrain, Xtest = X.loc[train_index], X.loc[test_index]
    ytrain, ytest = y[train_index], y[test_index]

    model = CatBoostRegressor(random_seed=34,bootstrap_type='Bayesian', max_depth=8,learning_rate=0.07,
                          iterations=4000,silent=True, early_stopping_rounds = 300)
    model.fit(Xtrain, ytrain, eval_set=[(Xtest,ytest)], verbose=200)
    
    pred = model.predict(Xtest)
    score = np.sqrt(mean_squared_error(ytest, pred))
    fold_score = fold_score + (score/n)
    print(fold_score)
    
    predictions = model.predict(test_df)
    test_pred.append(predictions)

0:	learn: 0.1849337	test: 0.1865800	best: 0.1865800 (0)	total: 92.5ms	remaining: 6m 9s
200:	learn: 0.0790287	test: 0.0885581	best: 0.0885581 (200)	total: 12.3s	remaining: 3m 52s
400:	learn: 0.0705442	test: 0.0868875	best: 0.0868834 (399)	total: 23.9s	remaining: 3m 34s
600:	learn: 0.0645392	test: 0.0858787	best: 0.0858760 (598)	total: 35.4s	remaining: 3m 20s
800:	learn: 0.0594231	test: 0.0857412	best: 0.0856489 (635)	total: 47.2s	remaining: 3m 8s
1000:	learn: 0.0551840	test: 0.0855220	best: 0.0854569 (947)	total: 58.7s	remaining: 2m 55s
1200:	learn: 0.0516340	test: 0.0851583	best: 0.0851583 (1200)	total: 1m 10s	remaining: 2m 44s
1400:	learn: 0.0485375	test: 0.0849381	best: 0.0848844 (1342)	total: 1m 22s	remaining: 2m 32s
1600:	learn: 0.0457683	test: 0.0849375	best: 0.0848649 (1453)	total: 1m 34s	remaining: 2m 21s
1800:	learn: 0.0432775	test: 0.0847430	best: 0.0847293 (1796)	total: 1m 46s	remaining: 2m 9s
2000:	learn: 0.0411158	test: 0.0849389	best: 0.0847293 (1796)	total: 1m 58s	remaini

Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.0865029287
bestIteration = 576

Shrink model to first 577 iterations.
0.1178075227905786
0:	learn: 0.1850668	test: 0.1852730	best: 0.1852730 (0)	total: 53.6ms	remaining: 3m 34s
200:	learn: 0.0790001	test: 0.0863291	best: 0.0863291 (200)	total: 11.9s	remaining: 3m 45s
400:	learn: 0.0706880	test: 0.0855534	best: 0.0854558 (373)	total: 23.9s	remaining: 3m 34s
600:	learn: 0.0646093	test: 0.0854192	best: 0.0853614 (593)	total: 35.3s	remaining: 3m 19s
800:	learn: 0.0595907	test: 0.0854903	best: 0.0853133 (613)	total: 47.2s	remaining: 3m 8s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.08531332824
bestIteration = 613

Shrink model to first 614 iterations.
0.12847168881762955
0:	learn: 0.1846692	test: 0.1920088	best: 0.1920088 (0)	total: 54.5ms	remaining: 3m 38s
200:	learn: 0.0786935	test: 0.0948170	best: 0.0948170 (200)	total: 13.1s	remaining: 4m 6s
400:	learn: 0.0701310	test: 0.0936642	best: 0.0935813

0.24754244683729387
0:	learn: 0.1850903	test: 0.1826470	best: 0.1826470 (0)	total: 76.9ms	remaining: 5m 7s
200:	learn: 0.0789114	test: 0.0906390	best: 0.0905740 (194)	total: 11.7s	remaining: 3m 41s
400:	learn: 0.0705453	test: 0.0899904	best: 0.0899904 (400)	total: 23.1s	remaining: 3m 27s
600:	learn: 0.0645252	test: 0.0896799	best: 0.0896750 (576)	total: 34.3s	remaining: 3m 14s
800:	learn: 0.0594512	test: 0.0894234	best: 0.0894234 (800)	total: 45.7s	remaining: 3m 2s
1000:	learn: 0.0553580	test: 0.0893488	best: 0.0892750 (853)	total: 57.4s	remaining: 2m 51s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.08927495811
bestIteration = 853

Shrink model to first 854 iterations.
0.2587018165809199
0:	learn: 0.1852167	test: 0.1810931	best: 0.1810931 (0)	total: 64ms	remaining: 4m 15s
200:	learn: 0.0791516	test: 0.0872188	best: 0.0872138 (197)	total: 11.5s	remaining: 3m 37s
400:	learn: 0.0708434	test: 0.0867763	best: 0.0865895 (307)	total: 23s	remaining: 3m 26s
600:	learn: 0

In [50]:
#from sklearn.model_selection import cross_val_predict

#y_pred = cross_val_predict(estimator=model, X= X y=y, cv=5)

In [51]:
df = pd.read_csv('SampleSubmission (1).csv')
df['Target'] = np.mean(test_pred, axis = 0)
df.to_csv('CatC.csv', index = False)

In [52]:
fold_score = 0
t_pred = []

n = 8
kf = StratifiedKFold(n)
kf = KFold(n_splits=25)
kf.get_n_splits(X)
  

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    Xtrain, Xtest = X.loc[train_index], X.loc[test_index]
    ytrain, ytest = y[train_index], y[test_index]

    model = CatBoostRegressor(random_seed=34,bootstrap_type='Bayesian', max_depth=10,learning_rate=0.2,
                          iterations=2000,silent=True, early_stopping_rounds = 300)
    model.fit(Xtrain, ytrain, eval_set=[(Xtest,ytest)], verbose=200)
    
    pred = model.predict(Xtest)
    score = np.sqrt(mean_squared_error(ytest, pred))
    fold_score = fold_score + (score/n)
    print(fold_score)
    
    predictions = model.predict(test_df)
    t_pred.append(predictions)

0:	learn: 0.1876031	test: 0.1892605	best: 0.1892605 (0)	total: 145ms	remaining: 4m 50s
200:	learn: 0.0796527	test: 0.0885328	best: 0.0885328 (200)	total: 26.8s	remaining: 3m 59s
400:	learn: 0.0705571	test: 0.0867467	best: 0.0867467 (400)	total: 53.4s	remaining: 3m 32s
600:	learn: 0.0642140	test: 0.0858922	best: 0.0858922 (600)	total: 1m 20s	remaining: 3m 7s
800:	learn: 0.0590328	test: 0.0855283	best: 0.0855283 (800)	total: 1m 47s	remaining: 2m 40s
1000:	learn: 0.0547120	test: 0.0852986	best: 0.0852542 (938)	total: 2m 14s	remaining: 2m 13s
1200:	learn: 0.0509072	test: 0.0851921	best: 0.0851688 (1142)	total: 2m 41s	remaining: 1m 47s
1400:	learn: 0.0476603	test: 0.0851165	best: 0.0851081 (1398)	total: 3m 7s	remaining: 1m 20s
1600:	learn: 0.0449038	test: 0.0851651	best: 0.0850969 (1429)	total: 3m 34s	remaining: 53.5s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.08509689908
bestIteration = 1429

Shrink model to first 1430 iterations.
0.010637112401123004
0:	learn: 0.

400:	learn: 0.0711045	test: 0.0876682	best: 0.0876682 (400)	total: 53.8s	remaining: 3m 34s
600:	learn: 0.0644213	test: 0.0874582	best: 0.0874433 (596)	total: 1m 20s	remaining: 3m 7s
800:	learn: 0.0593300	test: 0.0874712	best: 0.0873132 (706)	total: 1m 46s	remaining: 2m 39s
1000:	learn: 0.0550543	test: 0.0876968	best: 0.0873132 (706)	total: 2m 13s	remaining: 2m 13s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.08731322948
bestIteration = 706

Shrink model to first 707 iterations.
0.11827950971385935
0:	learn: 0.1875663	test: 0.1879101	best: 0.1879101 (0)	total: 132ms	remaining: 4m 23s
200:	learn: 0.0795881	test: 0.0871093	best: 0.0871093 (200)	total: 26.8s	remaining: 4m
400:	learn: 0.0709379	test: 0.0860479	best: 0.0860322 (398)	total: 52.9s	remaining: 3m 30s
600:	learn: 0.0643459	test: 0.0856123	best: 0.0855307 (566)	total: 1m 19s	remaining: 3m 4s
800:	learn: 0.0591947	test: 0.0856440	best: 0.0855307 (566)	total: 1m 46s	remaining: 2m 38s
Stopped by overfitting de

1000:	learn: 0.0546128	test: 0.0896000	best: 0.0894851 (789)	total: 2m 13s	remaining: 2m 13s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.08948507011
bestIteration = 789

Shrink model to first 790 iterations.
0.23797863802302868
0:	learn: 0.1876470	test: 0.1891009	best: 0.1891009 (0)	total: 155ms	remaining: 5m 10s
200:	learn: 0.0797780	test: 0.0841868	best: 0.0841868 (200)	total: 27s	remaining: 4m 1s
400:	learn: 0.0708538	test: 0.0822561	best: 0.0822438 (398)	total: 53.9s	remaining: 3m 34s
600:	learn: 0.0644327	test: 0.0816361	best: 0.0816091 (598)	total: 1m 20s	remaining: 3m 7s
800:	learn: 0.0592288	test: 0.0815969	best: 0.0815159 (771)	total: 1m 47s	remaining: 2m 41s
1000:	learn: 0.0547073	test: 0.0814394	best: 0.0814175 (987)	total: 2m 14s	remaining: 2m 13s
1200:	learn: 0.0510514	test: 0.0815792	best: 0.0814175 (987)	total: 2m 41s	remaining: 1m 47s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.08141747724
bestIteration = 987

Shrink mod

In [53]:
df = pd.read_csv('SampleSubmission (1).csv')
df['Target'] = np.mean(t_pred, axis = 0)
df.to_csv('CatCCC.csv', index = False)

In [54]:
import time
from sklearn import metrics
from sklearn.metrics import mean_squared_error,log_loss
from sklearn.model_selection import KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [94]:
param = {'objective':'regression','num_leaves':8,
         'learning_rate':0.02, 'n_estimators':10000,
         'max_bin': 55, "bagging_fraction": 0.2,
         "bagging_freq": 5, "feature_fraction": 0.4,
         "feature_fraction_seed":9, "bagging_seed":9,
         "min_data_in_leaf":6,
         "metric": 'rmse',
         "verbosity": 1,
         "verbose_eval": False,
        'early_stopping_rounds': 300}

In [95]:
import lightgbm as lgb

features = [c for c in train_df.columns if c not in ['Target']]
target = train_df['Target']

folds = KFold(n_splits=10, shuffle=True, random_state=15)
oof_lgb = np.zeros(len(train_df))
predictions_lgb = np.zeros(len(test_df))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 13000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 500)
    oof_lgb[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions_lgb += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
oof_lgb[oof_lgb < 0] = 0
print("CV score: {:<8.5f}".format(mean_squared_error(oof_lgb, target)**0.5))


fold n°0
Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 0.107053	valid_1's rmse: 0.10733


C:\Users\Osuntoki\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Osuntoki\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	training's rmse: 0.0978605	valid_1's rmse: 0.0984028
[300]	training's rmse: 0.0948461	valid_1's rmse: 0.0955402
[400]	training's rmse: 0.0931435	valid_1's rmse: 0.0941077
[500]	training's rmse: 0.0919596	valid_1's rmse: 0.0931301
[600]	training's rmse: 0.0910619	valid_1's rmse: 0.0923819
[700]	training's rmse: 0.0903109	valid_1's rmse: 0.0918779
[800]	training's rmse: 0.0895998	valid_1's rmse: 0.0915201
[900]	training's rmse: 0.0890143	valid_1's rmse: 0.0912944
[1000]	training's rmse: 0.0884739	valid_1's rmse: 0.0909413
[1100]	training's rmse: 0.0879637	valid_1's rmse: 0.0907315
[1200]	training's rmse: 0.0874286	valid_1's rmse: 0.0904148
[1300]	training's rmse: 0.0869918	valid_1's rmse: 0.0901538
[1400]	training's rmse: 0.0866261	valid_1's rmse: 0.089935
[1500]	training's rmse: 0.0862279	valid_1's rmse: 0.0897676
[1600]	training's rmse: 0.0858625	valid_1's rmse: 0.0895459
[1700]	training's rmse: 0.0855089	valid_1's rmse: 0.0895002
[1800]	training's rmse: 0.0851634	valid_1's rmse:

[3600]	training's rmse: 0.0802199	valid_1's rmse: 0.0928292
Early stopping, best iteration is:
[3396]	training's rmse: 0.080609	valid_1's rmse: 0.0927145
fold n°3
Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 0.106999	valid_1's rmse: 0.108334
[200]	training's rmse: 0.0979165	valid_1's rmse: 0.0989767
[300]	training's rmse: 0.095013	valid_1's rmse: 0.0960152
[400]	training's rmse: 0.0932724	valid_1's rmse: 0.0944846
[500]	training's rmse: 0.0919816	valid_1's rmse: 0.0933899
[600]	training's rmse: 0.091067	valid_1's rmse: 0.0927336
[700]	training's rmse: 0.090273	valid_1's rmse: 0.0922734
[800]	training's rmse: 0.0895917	valid_1's rmse: 0.0918203
[900]	training's rmse: 0.0889877	valid_1's rmse: 0.0914981
[1000]	training's rmse: 0.0884425	valid_1's rmse: 0.0912467
[1100]	training's rmse: 0.0879443	valid_1's rmse: 0.0909303
[1200]	training's rmse: 0.0874534	valid_1's rmse: 0.0907251
[1300]	training's rmse: 0.0869892	valid_1's rmse: 0.0905398
[1400]	tr

[2700]	training's rmse: 0.0824493	valid_1's rmse: 0.0898696
[2800]	training's rmse: 0.0822148	valid_1's rmse: 0.089958
Early stopping, best iteration is:
[2546]	training's rmse: 0.0828364	valid_1's rmse: 0.0897917
fold n°7
Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 0.106834	valid_1's rmse: 0.108365
[200]	training's rmse: 0.0976058	valid_1's rmse: 0.0998723
[300]	training's rmse: 0.0945468	valid_1's rmse: 0.0976631
[400]	training's rmse: 0.0928986	valid_1's rmse: 0.0965758
[500]	training's rmse: 0.091664	valid_1's rmse: 0.095851
[600]	training's rmse: 0.0907595	valid_1's rmse: 0.0952874
[700]	training's rmse: 0.0899642	valid_1's rmse: 0.0948246
[800]	training's rmse: 0.0893448	valid_1's rmse: 0.0945629
[900]	training's rmse: 0.0887416	valid_1's rmse: 0.0943277
[1000]	training's rmse: 0.088157	valid_1's rmse: 0.0940035
[1100]	training's rmse: 0.0876781	valid_1's rmse: 0.0936583
[1200]	training's rmse: 0.0872167	valid_1's rmse: 0.0934405
[1300]	tr

In [93]:
df = pd.read_csv('SampleSubmission (1).csv')
df['Target'] = predictions_lgb
df.to_csv('LGBM.csv', index = False)

In [96]:
param = {'num_leaves': 29, ## using 29 and 30
     'min_data_in_leaf': 50, 
     'objective':'regression',
     'max_depth': 18,
     'learning_rate': 0.006,
     "boosting": "gbdt",
     "feature_fraction": 0.2,
     "bagging_freq": 0,
     "bagging_fraction": 1.0 ,
     "bagging_seed": 4,
     "metric": 'rmse',
     "verbosity": 1,
     "verbose_eval": False,
    "early_stopping_rounds" : 300
    }

In [97]:
import time

In [98]:
import lightgbm as lgb

features = [c for c in train_df.columns if c not in ['Target']]
target = train_df['Target']

folds = KFold(n_splits=10, shuffle=True, random_state=15)
oof_lgb = np.zeros(len(train_df))
predictions_lgb = np.zeros(len(test_df))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 13000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 500)
    oof_lgb[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions_lgb += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
oof_lgb[oof_lgb < 0] = 0
print("CV score: {:<8.5f}".format(mean_squared_error(oof_lgb, target)**0.5))


fold n°0
Training until validation scores don't improve for 300 rounds


C:\Users\Osuntoki\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 0.14661	valid_1's rmse: 0.147758
[200]	training's rmse: 0.123237	valid_1's rmse: 0.124645
[300]	training's rmse: 0.111497	valid_1's rmse: 0.113291
[400]	training's rmse: 0.104883	valid_1's rmse: 0.107037
[500]	training's rmse: 0.100622	valid_1's rmse: 0.103078
[600]	training's rmse: 0.097559	valid_1's rmse: 0.100287
[700]	training's rmse: 0.0952818	valid_1's rmse: 0.0982746
[800]	training's rmse: 0.0934468	valid_1's rmse: 0.0967272
[900]	training's rmse: 0.0919156	valid_1's rmse: 0.0954864
[1000]	training's rmse: 0.0906328	valid_1's rmse: 0.0944547
[1100]	training's rmse: 0.0895783	valid_1's rmse: 0.0936308
[1200]	training's rmse: 0.0886572	valid_1's rmse: 0.0929765
[1300]	training's rmse: 0.0878594	valid_1's rmse: 0.0924175
[1400]	training's rmse: 0.0871481	valid_1's rmse: 0.0919256
[1500]	training's rmse: 0.0865106	valid_1's rmse: 0.0915031
[1600]	training's rmse: 0.0859233	valid_1's rmse: 0.0911367
[1700]	training's rmse: 0.0853804	valid_1's rmse: 0.0908071
[1

[700]	training's rmse: 0.0949402	valid_1's rmse: 0.101843
[800]	training's rmse: 0.0930744	valid_1's rmse: 0.100371
[900]	training's rmse: 0.0915298	valid_1's rmse: 0.0991793
[1000]	training's rmse: 0.0902387	valid_1's rmse: 0.0982075
[1100]	training's rmse: 0.0891736	valid_1's rmse: 0.0974965
[1200]	training's rmse: 0.0882585	valid_1's rmse: 0.0968992
[1300]	training's rmse: 0.0874434	valid_1's rmse: 0.0964225
[1400]	training's rmse: 0.0867241	valid_1's rmse: 0.0960254
[1500]	training's rmse: 0.0860781	valid_1's rmse: 0.0957025
[1600]	training's rmse: 0.0854928	valid_1's rmse: 0.0954049
[1700]	training's rmse: 0.0849518	valid_1's rmse: 0.0951068
[1800]	training's rmse: 0.0844401	valid_1's rmse: 0.0948547
[1900]	training's rmse: 0.0839562	valid_1's rmse: 0.0946388
[2000]	training's rmse: 0.0835028	valid_1's rmse: 0.09444
[2100]	training's rmse: 0.0830796	valid_1's rmse: 0.0942664
[2200]	training's rmse: 0.0826864	valid_1's rmse: 0.0941167
[2300]	training's rmse: 0.0823295	valid_1's rms

[300]	training's rmse: 0.11154	valid_1's rmse: 0.113038
[400]	training's rmse: 0.104873	valid_1's rmse: 0.106962
[500]	training's rmse: 0.100559	valid_1's rmse: 0.103148
[600]	training's rmse: 0.0974572	valid_1's rmse: 0.100509
[700]	training's rmse: 0.0951573	valid_1's rmse: 0.0986452
[800]	training's rmse: 0.0933029	valid_1's rmse: 0.0971902
[900]	training's rmse: 0.0917382	valid_1's rmse: 0.0960331
[1000]	training's rmse: 0.0904159	valid_1's rmse: 0.0951107
[1100]	training's rmse: 0.0893327	valid_1's rmse: 0.0944099
[1200]	training's rmse: 0.0883804	valid_1's rmse: 0.0938174
[1300]	training's rmse: 0.0875451	valid_1's rmse: 0.0933445
[1400]	training's rmse: 0.0867948	valid_1's rmse: 0.0929657
[1500]	training's rmse: 0.0861353	valid_1's rmse: 0.0926692
[1600]	training's rmse: 0.0855301	valid_1's rmse: 0.0924192
[1700]	training's rmse: 0.0849763	valid_1's rmse: 0.0922037
[1800]	training's rmse: 0.0844595	valid_1's rmse: 0.0920334
[1900]	training's rmse: 0.0839702	valid_1's rmse: 0.091

[3200]	training's rmse: 0.0799434	valid_1's rmse: 0.0888027
[3300]	training's rmse: 0.0796847	valid_1's rmse: 0.0887614
[3400]	training's rmse: 0.0794269	valid_1's rmse: 0.0887354
[3500]	training's rmse: 0.0791821	valid_1's rmse: 0.0887287
[3600]	training's rmse: 0.0789454	valid_1's rmse: 0.0887275
[3700]	training's rmse: 0.0787171	valid_1's rmse: 0.0887046
[3800]	training's rmse: 0.0784882	valid_1's rmse: 0.0886798
[3900]	training's rmse: 0.0782668	valid_1's rmse: 0.0886645
[4000]	training's rmse: 0.0780512	valid_1's rmse: 0.0886655
[4100]	training's rmse: 0.0778355	valid_1's rmse: 0.0886705
[4200]	training's rmse: 0.0776299	valid_1's rmse: 0.0886766
Early stopping, best iteration is:
[3920]	training's rmse: 0.078221	valid_1's rmse: 0.0886587
fold n°7
Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 0.146581	valid_1's rmse: 0.148723
[200]	training's rmse: 0.12318	valid_1's rmse: 0.125477
[300]	training's rmse: 0.111415	valid_1's rmse: 0.114164
[400]

[1700]	training's rmse: 0.0850367	valid_1's rmse: 0.092757
[1800]	training's rmse: 0.0845235	valid_1's rmse: 0.0925227
[1900]	training's rmse: 0.0840568	valid_1's rmse: 0.092349
[2000]	training's rmse: 0.0836179	valid_1's rmse: 0.0921756
[2100]	training's rmse: 0.0831915	valid_1's rmse: 0.09201
[2200]	training's rmse: 0.0827929	valid_1's rmse: 0.0918763
[2300]	training's rmse: 0.0824245	valid_1's rmse: 0.0917715
[2400]	training's rmse: 0.0820599	valid_1's rmse: 0.0916712
[2500]	training's rmse: 0.0817029	valid_1's rmse: 0.091578
[2600]	training's rmse: 0.0813609	valid_1's rmse: 0.0915032
[2700]	training's rmse: 0.0810446	valid_1's rmse: 0.0914426
[2800]	training's rmse: 0.0807254	valid_1's rmse: 0.091378
[2900]	training's rmse: 0.0804219	valid_1's rmse: 0.0913197
[3000]	training's rmse: 0.0801323	valid_1's rmse: 0.0912695
[3100]	training's rmse: 0.079848	valid_1's rmse: 0.0912124
[3200]	training's rmse: 0.0795775	valid_1's rmse: 0.0911706
[3300]	training's rmse: 0.0793163	valid_1's rms

In [99]:
sub= ttest[["ID"]]
sub['Target'] = predictions_lgb
sub.to_csv('postchallengeLtt.csv', index = False)

C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [101]:
import xgboost
from xgboost import XGBRegressor

In [125]:
fold_score = 0
t_pred = []

n = 8
kf = StratifiedKFold(n)
kf = KFold(n_splits=8)
kf.get_n_splits(X)
  

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    Xtrain, Xtest = X.loc[train_index], X.loc[test_index]
    ytrain, ytest = y[train_index], y[test_index]

    model = XGBRegressor(random_seed=34,bootstrap_type='Bayesian', max_depth=10, eta=0.004, eval_metric ='rmse',
                          iterations=2000,subsample=0.7, colsample_bytree=0.8,silent=False, early_stopping_rounds = 300)
    model.fit(Xtrain, ytrain, eval_set=[(Xtest,ytest)], verbose=200)
    
    pred = model.predict(Xtest)
    score = np.sqrt(mean_squared_error(ytest, pred))
    fold_score = fold_score + (score/n)
    print(fold_score)
    
    predictions = model.predict(test_df)
    t_pred.append(predictions)

[21:31:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { bootstrap_type, early_stopping_rounds, iterations, random_seed, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.24322
[99]	validation_0-rmse:0.17918
0.022397632042570288
[21:31:21] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { bootstrap_type, early_stopping_rounds, iterations, random_seed, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:

In [ ]:
import random
classifier = XGBRegressor(n_estimators = 2000, objective = "reg:squarederror")
# Create Hyperparameter Search Space

param_dist = {
    "learning_rate": [0.01, 0.03, 0.05,0.08,0.1,0.2],
    "gamma": [0.5, 0.7,0.9],
    "max_depth": [5,7,8,9,10],
    "min_child_weight": [1, 2, 3,5,7],
    "max_delta_step":[0,1,2,3],
    "reg_lambda":[0.3,0.5,1],
    "subsample":[0.3,0.5,0.7,0.9],
    "colsample_bytree":[0.3,0.4,0.6,0.8]
}

# create random search
# Create randomized search 5-fold cross validation and 100 iterations
clf = RandomizedSearchCV(
 estimator=classifier,
 param_distributions=param_dist,
 random_state=1,
 n_iter=500,
 cv=5,
 verbose=0,
 n_jobs=-1,
)
# Fit randomized search
best_model = clf.fit(X, y)

In [ ]:
dat = pd.read_csv('postchallengeLt.csv')
dat1 = pd.read_csv('LGBM.csv')
dat2 = pd.read_csv('CatC.csv')
dat3 = pd.read_csv('final_sub.csv')
dat4 = pd.read_csv('postchallenge.csv')
dat5 = pd.read_csv('n1.csv')

In [123]:
#datt1 = (dat1['Target'] + dat2['Target'])/2
datt2 = (dat2['Target'] + dat3['Target'])/2
datt3 = (dat['Target'] + dat2['Target'] + dat3['Target']+dat5['Target'])/4

In [114]:
df = pd.read_csv('SampleSubmission (1).csv')
df['Target'] = datt1
df.to_csv('FL1.csv', index = False)

In [115]:
df['Target'] = datt2
df.to_csv('FL2.csv', index = False)

In [124]:
df['Target'] = datt3
df.to_csv('FL3.csv', index = False)

In [70]:
dat5 = pd.read_csv('FL2.csv')
dat6 = pd.read_csv('FL3.csv')
dat7 = pd.read_csv('FL4.csv')
dat8 = pd.read_csv('FL5.csv')
dat9 = pd.read_csv('FL7.csv')
dat10 = pd.read_csv('FL1.csv')

In [112]:
datt4 = (dat['Target']+dat1['Target']+dat3['Target']+dat4['Target']+dat5['Target'])/5

sub= ttest[["ID"]]
sub['Target'] = datt4
sub.to_csv('final_.csv', index = False)

C:\Users\Osuntoki\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [72]:
datt5 = (dat1['Target'] + dat2['Target'] + dat4['Target']+dat5['Target']+dat6['Target'])/5

In [73]:
df['Target'] = datt4
df.to_csv('F4.csv', index = False)

In [74]:
df['Target'] = datt5
df.to_csv('FL5.csv', index = False)

In [75]:
datt7 = (dat1['Target'] + dat2['Target'] + dat3['Target']+dat7['Target'])/4

In [76]:
df['Target'] = datt7
df.to_csv('FL7.csv', index = False)

In [77]:
datt8 = (dat1['Target'] + dat2['Target'] +dat5['Target']+ dat6['Target']+ dat7['Target']+ 
         dat8['Target']+ dat9['Target']+ dat10['Target'])/8

In [78]:
df['Target'] = datt8
df.to_csv('FL8.csv', index = False)

In [79]:
datt9 = (dat1['Target'] + dat2['Target'] + dat3['Target'] + dat4['Target'] + dat5['Target'] + 
         dat6['Target'] + dat7['Target'] + dat8['Target'] + dat9['Target'] + dat10['Target'])/10

In [80]:
df['Target'] = datt9
df.to_csv('FL9.csv', index = False)

In [81]:
dtt1 = 0.38*(dat1['Target']) + 0.35*(dat2['Target']) + 0.22*(dat3['Target']) + 0.05*(dat4['Target'])

In [82]:
df['Target'] = dtt1
df.to_csv('n1.csv', index = False)

In [83]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [84]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    rmse= np.sqrt(-cross_val_score(model, X.values, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [85]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [86]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [87]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [88]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [89]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


In [90]:
model_lgb = lgb.LGBMRegressor(**param)

In [91]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1014 (0.0014)



In [92]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1014 (0.0013)



In [93]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 0.0877 (0.0011)



In [94]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[10:51:49] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:52:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:52:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [95]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

LGBM score: 0.1473 (0.0023)



In [96]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models_])
        return np.mean(predictions, axis=1)  

In [97]:
averaged_models = AveragingModels(models = (ENet, GBoost, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.0935 (0.0012)



In [98]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1) for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [99]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

C:\Users\Osuntoki\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Osuntoki\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "<ipython-input-98-1b2964bdcbe6>", line 20, in fit
    instance.fit(X[train_index], y[train_index])
  File "C:\Users\Osuntoki\anaconda3\lib\site-packages\pandas\core\series.py", line 877, in __getitem__
    return self._get_with(key)
  File "C:\Users\Osuntoki\anaconda3\lib\site-packages\pandas\core\series.py", line 912, in _get_with
    return self.loc[key]
  File "C:\Users\Osuntoki\anaconda3\lib\site-packages\pandas\core\indexing.py", line 895, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "C:\Users\Osuntoki\anaconda3\lib\sit

Stacking Averaged models score: nan (nan)


In [100]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [101]:
stacked_averaged_models.fit(X.values, y)
stacked_train_pred = stacked_averaged_models.predict(X.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test_df.values))
print(rmsle(y, stacked_train_pred))

0.06889832667634832


In [102]:
stacked_pred 

array([0.16644971, 0.17563372, 0.96371922, ..., 0.54671661, 0.23441609,
       0.39930728])

In [103]:
model_xgb.fit(X, y)
xgb_train_pred = model_xgb.predict(X)
xgb_pred = np.expm1(model_xgb.predict(test_df))
print(rmsle(y, xgb_train_pred))

[11:20:59] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.08141699609082433


In [105]:
model_lgb.fit(X, y)
lgb_train_pred = model_lgb.predict(X)
lgb_pred = np.expm1(model_lgb.predict(test_df.values))
print(rmsle(y, lgb_train_pred))

0.14673680506531425


In [106]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y,stacked_train_pred*0.9 +
               xgb_train_pred*0.1))

RMSLE score on train data:
0.06995797561027817


In [109]:
ensemble = stacked_pred*0.6 + xgb_pred*0.4

In [110]:
df['Target'] = ensemble
df.to_csv('submissionyy.csv', index = False)